In [53]:
import pandas as pd
from pandas.api.types import CategoricalDtype

In [50]:
df_gasoline = pd.read_csv("./01_raw_data/EV_project/에너지·연료비용/휘발유 가격/휘발유.csv", skiprows=6, encoding='cp949')
df_gasoline_avg = pd.read_csv("./01_raw_data/EV_project/에너지·연료비용/휘발유 가격/휘발유_전국.csv", skiprows=6 , encoding='cp949')
df_diesel = pd.read_csv("./01_raw_data/EV_project/에너지·연료비용/경유 가격/디젤.csv" , skiprows=6, encoding='cp949')
df_diesel_avg = pd.read_csv("./01_raw_data/EV_project/에너지·연료비용/경유 가격/디젤_전국.csv", skiprows=6 , encoding='cp949')

In [51]:
print(df_diesel.columns.tolist())

['구분', '서울', '부산', '대구', '인천', '광주', '대전', '울산', '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주', '세종']


In [52]:
# 2. '구분' → '기간'으로 열 이름 통일
for df in [df_gasoline, df_gasoline_avg, df_diesel, df_diesel_avg]:
    df.rename(columns={df.columns[0]: '기간'}, inplace=True)

# 3. long-form 변환 함수 정의
def reshape_long(df, 전국=False):
    if 전국:
        df['시도'] = '전국'
        df = df.rename(columns={df.columns[1]: '가격'})
        df_long = df[['기간', '시도', '가격']]
    else:
        df_long = df.melt(id_vars='기간', var_name='시도', value_name='가격')

    # 연도/월 추출
    df_long['연도'] = df_long['기간'].str[:4].astype(int)
    df_long['월'] = df_long['기간'].str[5:7].astype(int)
    df_long['가격'] = pd.to_numeric(df_long['가격'], errors='coerce')

    return df_long[['연도', '월', '시도', '가격']]

# 4. 변환 수행
gasoline_long = reshape_long(df_gasoline)
gasoline_avg_long = reshape_long(df_gasoline_avg, 전국=True)
diesel_long = reshape_long(df_diesel)
diesel_avg_long = reshape_long(df_diesel_avg, 전국=True)

# 5. 결과 확인
print("휘발유 (지역별):")
print(gasoline_long.head())

print("\n휘발유 (전국):")
print(gasoline_avg_long.head())

print("\n디젤 (지역별):")
print(diesel_long.head())

print("\n디젤 (전국):")
print(diesel_avg_long.head())

휘발유 (지역별):
     연도   월  시도       가격
0  2015   6  서울  1670.46
1  2015   7  서울  1664.34
2  2015   8  서울  1635.58
3  2015   9  서울  1601.05
4  2015  10  서울  1582.03

휘발유 (전국):
     연도   월  시도       가격
0  2015   7  전국  1922.84
1  2015   8  전국  1896.98
2  2015   9  전국  1866.32
3  2015  10  전국  1854.83
4  2015  11  전국  1829.07

디젤 (지역별):
     연도   월  시도       가격
0  2015   6  서울  1458.61
1  2015   7  서울  1445.31
2  2015   8  서울  1409.95
3  2015   9  서울  1376.38
4  2015  10  서울  1359.41

디젤 (전국):
     연도   월  시도       가격
0  2015   7  전국  1354.65
1  2015   8  전국  1307.77
2  2015   9  전국  1263.79
3  2015  10  전국  1251.92
4  2015  11  전국  1234.95


C:\Users\junjo\AppData\Local\Temp\ipykernel_74184\3033525121.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_long['연도'] = df_long['기간'].str[:4].astype(int)
C:\Users\junjo\AppData\Local\Temp\ipykernel_74184\3033525121.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_long['월'] = df_long['기간'].str[5:7].astype(int)
C:\Users\junjo\AppData\Local\Temp\ipykernel_74184\3033525121.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [63]:
from pandas.api.types import CategoricalDtype

# 1. 시도 순서에 '전국' 포함
sido_order = [
    '서울', '부산', '대구', '인천', '광주', '대전', '울산', '경기', '강원',
    '충북', '충남', '전북', '전남', '경북', '경남', '제주', '세종', '전국'
]
sido_type = CategoricalDtype(categories=sido_order, ordered=True)

# 2. 전국 포함해서 범주형 지정 (병합 전에 반드시 실행)
for df in [gasoline_long, gasoline_avg_long, diesel_long, diesel_avg_long]:
    df['시도'] = df['시도'].astype(str)  # ← 혹시 object로 되어있을 수도 있음
    df['시도'] = df['시도'].astype(sido_type)

# 3. concat
gasoline_all = pd.concat([gasoline_long, gasoline_avg_long], ignore_index=True)
diesel_all = pd.concat([diesel_long, diesel_avg_long], ignore_index=True)

# 4. 병합
merged = pd.merge(
    gasoline_all,
    diesel_all,
    on=['연도', '월', '시도'],
    how='outer',  # ← outer로 하면 누락 안 됨
    suffixes=('_휘발유', '_디젤')
)

# 5. 열 이름 정리
merged = merged[['연도', '월', '시도', '가격_휘발유', '가격_디젤']]
merged = merged.rename(columns={'가격_휘발유': '휘발유가격', '가격_디젤': '디젤가격'})

# 6. 정렬
merged = merged.sort_values(['연도', '월', '시도']).reset_index(drop=True)

# 결과 확인
merged


,연도,월,시도,휘발유가격,디젤가격
0,2015,6,서울,1670.46,1458.61
1,2015,6,부산,1571.07,1362.94
2,2015,6,대구,1569.50,1358.55
3,2015,6,인천,1580.86,1365.15
4,2015,6,광주,1564.17,1354.82
...,...,...,...,...,...
2172,2025,6,경북,1631.16,1491.00
2173,2025,6,경남,1631.26,1493.62
2174,2025,6,제주,1697.45,1557.12
2175,2025,6,세종,1629.31,1500.88


In [64]:
merged.to_csv("./02_interim_data/oil_prices.csv", index=False, encoding='utf-8-sig')